In [ ]:
### Imports from algo_common

import csv
import datetime
from dateutil import tz
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, FormatStrFormatter, AutoMinorLocator
import numpy as np
import os
import pandas as pd
import pytz
import sys
import time

os.chdir(r'D:\algo2\IBridgePy_Win_Anaconda38_64 - Copy')
sys.path.append(os.path.join(os.getcwd(), '..'))

from trader_factory import build_active_IBridgePy_plus
from IBridgePy.IbridgepyTools import symbol

from Utils import Algos
from Utils import EnterExits
from Utils import IndicatorCalculations
from Utils import Orders
from Utils import Plotters
from Utils import Utils
from UnitTests import testutils








In [ ]:
sim = build_active_IBridgePy_plus('DU2628075') # Build an active trader to get connected to Interactive Brokers 

In [ ]:
def plotDailyCandleStick(priceData, 
                 logData, 
                 emaList: list = ['fast', 'slow', "macd"],
                 saveFig: bool = False,
                 outPutFileName: str=r"C:\Users\Lane Capital\OneDrive\Desktop\repos\plots\TestCandle.png",
                 x = None):
    
    """This function takes in a historical input data dataframe, EMA dataframe from calcEMAVector 
    and plots the differnet EMAS against the price. Price is assumed to be the close price"""
    # TODO add volume held for plotting
    #create figure
    fig, ax = plt.subplots(figsize=(36,28))

    #define width of candlestick elements
    width = 0.0005
    width2 = 0.0001

    #define up and down priceData
    up = priceData[priceData.close>=priceData.open]
    down = priceData[priceData.close<priceData.open]
    log = logData.copy()
    if x!=None:
    # remove elements not in the x limit
        up = up[up.time>datetime.datetime.timestamp(x[0])]
        up = up[up.time<datetime.datetime.timestamp(x[1])]

        down = down[down.time>datetime.datetime.timestamp(x[0])]
        down = down[down.time<datetime.datetime.timestamp(x[1])]

        log = log[log.time>datetime.datetime.timestamp(x[0])]
        log = log[log.time<datetime.datetime.timestamp(x[1])]

    #define colors to use
    col1 = 'green'
    col2 = 'red'

    for i,j in enumerate(emaList):
        exec('temp{}=log.{}'.format(i,j))    
        exec('ax.plot(log.date, temp{})'.format(i))

#     outArray=Plotters.plotEntryStatus(log)
#     ax2 = ax.twinx()
#     ax2.plot(log.date,outArray,color='r')
#     fig.legend(['fast','slow','holding flag'],fontsize=30) # sticking this code here as a lazy fix for the legend

    #plot up priceData
    ax.bar(up.date,up.close-up.open,width,bottom=up.open,color=col1)
    ax.bar(up.date,up.high-up.close,width2,bottom=up.close,color=col1)
    ax.bar(up.date,up.low-up.open,width2,bottom=up.open,color=col1)

    # #plot down priceData
    ax.bar(down.date,down.close-down.open,width,bottom=down.open,color=col2)
    ax.bar(down.date,down.high-down.open,width2,bottom=down.open,color=col2)
    ax.bar(down.date,down.low-down.close,width2,bottom=down.close,color=col2)
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%H:%M',tz=tz.gettz('America/New York')))


    # Turn grid on for both major and minor ticks and style minor slightly differently.
    ax.grid(which='major')
    ax.grid(which='minor', linewidth=0.25)
    ax.minorticks_on()
    plt.ylim(np.min([np.min(up.low),np.min(down.low)])-.05,np.max([np.max(up.high),np.max(down.high)])+.05)
    if x==None:
        plt.xlabel("Time",fontsize=30)
        plt.ylabel("Stock Price (USD)",fontsize=30)
        plt.title("SPY Stock Price: "+str(datetime.date.today()),fontdict={'fontsize': 40})
    else:
        ax.set(xlim = x)
        plt.xlabel("Time",fontsize=30)
        plt.ylabel("Stock Price (USD)",fontsize=30)
        plt.title("SPY Stock Price: "+str(datetime.date.today()),fontdict={'fontsize': 40})
           
    outArray=Plotters.plotEntryStatus(log)
    ax2 = ax.twinx()
    ax2.plot(log.date,outArray,color='r')
    ax2.set(ylim = (-1,1))
    if saveFig:
        fig.savefig(outPutFileName)
        
# plotDailyCandleStick(priceData, logData, emaList = ['fast', 'slow'])

In [ ]:
def generateDailyPlots():
    # create new directory for output plots
    newDir = r"D:\algo2\IBridgePy_Win_Anaconda38_64 - Copy\Output\candlestick_"+str(datetime.date.today())
    if not os.path.isdir(newDir):
        os.mkdir(newDir)

    # load price data
    priceName = r"log\Daily_Price_Data_"+str(datetime.date.today())+".csv"
    priceColumns = ["date", "time", "open", "high", "low", "close", "volume"]
    priceData = pd.read_csv(priceName, header=1, names=priceColumns)
    # convert the time into datetime
    priceData['date'] =  np.array([datetime.datetime.fromtimestamp(f) for f in priceData.time.values])

    # load ema data
    logName = r"log\Daily_Log_"+str(datetime.date.today())+".csv"
    logColumns = ["time", "fast", "slow", "tLine", "macd", "signalLine", "longBool", "shortBool", 
                "enterFlag", "entryFlag", "priceAsk", "priceBid", "priceLast", "priceMid"]
    logData = pd.read_csv(logName, header=1, names=logColumns)
    # convert the time into datetime
    logData['date'] = np.array([datetime.datetime.fromtimestamp(float(f)) for f in logData.time.values])

    # define x limits to plot hourly data
    xlim = [(datetime.datetime(2022,11,11,7,31,0),datetime.datetime(2022,11,11,8,31,0)),
            (datetime.datetime(2022,11,11,8,31,0),datetime.datetime(2022,11,11,9,31,0)),
            (datetime.datetime(2022,11,11,9,31,0),datetime.datetime(2022,11,11,10,31,0)),
            (datetime.datetime(2022,11,11,10,31,0),datetime.datetime(2022,11,11,11,31,0)),
            (datetime.datetime(2022,11,11,11,31,0),datetime.datetime(2022,11,11,12,31,0)),
            (datetime.datetime(2022,11,11,12,31,0),datetime.datetime(2022,11,11,13,31,0)),
            (datetime.datetime(2022,11,11,13,31,0),datetime.datetime(2022,11,11,14,31,0)),
            (datetime.datetime(2022,11,11,14,31,0),datetime.datetime(2022,11,11,15,31,0)),
            (datetime.datetime(2022,11,11,15,31,0),datetime.datetime(2022,11,11,16,1,0))]

    # plot the full days plot
    outputFileName = newDir+"\\DailyPlot"+str(datetime.date.today())+"_full"+".png"
    Plotters.plotDailyCandleStickMacd(priceData, logData, emaList = ['fast', 'slow'], outPutFileName=outputFileName,saveFig=True)

    # generate hourly plots
    for x in xlim:
        outputFileName = newDir+"\\DailyPlot"+str(datetime.date.today())+"_"+str(x[0].hour)+".png"
        Plotters.plotDailyCandleStickMacd(priceData, logData, emaList = ['fast', 'slow'], x=x,outPutFileName=outputFileName,saveFig=True)

In [ ]:
def plotDailyCandleStickMacd(priceData, 
                 logData, 
                 emaList: list = ['fast', 'slow', "macd"],
                 saveFig: bool = False,
                 outPutFileName: str=r"C:\Users\Lane Capital\OneDrive\Desktop\repos\plots\TestCandle.png",
                 x = None):

    """This function takes in a historical input data dataframe, EMA dataframe from calcEMAVector 
    and plots the differnet EMAS against the price. Price is assumed to be the close price"""

    #create figure
    fig, (ax,ax1) = plt.subplots(2,figsize=(36,28),gridspec_kw={'height_ratios': [2, 1]})

    #define width of candlestick elements
    width = 0.0005
    width2 = 0.0001

    #define up and down priceData
    up = priceData[priceData.close>=priceData.open]
    down = priceData[priceData.close<priceData.open]
    log = logData.copy()
    if x!=None:
        # remove elements not in the x limit
        up = up[up.time>datetime.datetime.timestamp(x[0])]
        up = up[up.time<datetime.datetime.timestamp(x[1])]

        down = down[down.time>datetime.datetime.timestamp(x[0])]
        down = down[down.time<datetime.datetime.timestamp(x[1])]

        log = log[log.time.values>datetime.datetime.timestamp(x[0])]
        log = log[log.time.values<datetime.datetime.timestamp(x[1])]

    #define colors to use
    col1 = 'green'
    col2 = 'red'



    outArray=Plotters.plotEntryStatus(log)
    ax1.plot(log.date,outArray)

    ax1.plot(log.date,log.macd)
    ax1.plot(log.date,log.signalLine)


    #plot up priceData
    ax.bar(up.date,up.close-up.open,width,bottom=up.open,color=col1)
    ax.bar(up.date,up.high-up.close,width2,bottom=up.close,color=col1)
    ax.bar(up.date,up.low-up.open,width2,bottom=up.open,color=col1)

    # #plot down priceData
    ax.bar(down.date,down.close-down.open,width,bottom=down.open,color=col2)
    ax.bar(down.date,down.high-down.open,width2,bottom=down.open,color=col2)
    ax.bar(down.date,down.low-down.close,width2,bottom=down.close,color=col2)
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%H:%M',tz=tz.gettz('America/New York')))
    
    ax.plot(log.date,log.fast)
    ax.plot(log.date,log.slow)

    # Turn grid on for both major and minor ticks and style minor slightly differently.
    ax.grid(which='major')
    ax.grid(which='minor', linewidth=0.25)
    ax.minorticks_on()
    ax.set_ylim(np.min([np.min(up.low),np.min(down.low)])-.05,np.max([np.max(up.high),np.max(down.high)])+.05)

    ax1.grid(which='major')
    ax1.grid(which='minor', linewidth=0.25)
    ax1.minorticks_on()
    ax1.set_ylim((-1.1,1.1))


    ax.set_xlabel("Time",fontsize=30)
    ax1.set_xlabel("Time",fontsize=30)
    ax.set_ylabel("Stock Price (USD)",fontsize=30)
    ax1.set_ylabel("Value (nd)",fontsize=30)
    ax.set_title("SPY Stock Price: "+str(datetime.date.today()),fontdict={'fontsize': 40})
    ax1.set_title("MacD/Signal Line Value",fontdict={'fontsize': 40})
    ax1.legend(["macd","signal line","holding flag"],fontsize=20)
    if x!=None:
        ax.set_xlim(x)
        ax1.set_xlim(x)

    if saveFig:
        fig.savefig(outPutFileName)

In [ ]:
def cleanLogData(logData):
    """ This function takes in the log data and cleans it up so it can be used for plotting and other tasks"""
    logData = logData[logData["time"]!="time"] # remove columns which result from rerunning algo in the day
    for i in range(len(logData.time)): # convert time to floating point
        logData.time.iloc[i]=float(logData.time.iloc[i])
        logData.fast.iloc[i]=float(logData.fast.iloc[i])
        logData.slow.iloc[i]=float(logData.slow.iloc[i])
        logData.macd.iloc[i]=float(logData.macd.iloc[i])
        logData.signalLine.iloc[i]=float(logData.signalLine.iloc[i])
    return logData

In [ ]:
def getPlottingHourLimits(logData):
    """This function takes in logData and outputs a list of tuples of datetimes to be used as the x axis in plots"""
    
    # get the unique hours in the log data
    dates = np.unique(np.array([datetime.datetime.fromtimestamp(logData.time.values[i]).hour for i in range(len(logData.time))]))
    
    # get todays date time
    today=datetime.datetime.today()
    
    # get list of tuples with monotonically increasing unique hours
    xlim = [(datetime.datetime(today.year,today.month,11,dates[i],31,0), datetime.datetime(today.year,today.month,11,dates[i+1],31,0)) for i in range(len(dates)-1)]
    
    return xlim